In [79]:
import numpy as np 
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import ColumnTransformer
import string
from sklearn.linear_model  import LogisticRegression
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import pymorphy2
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\baben_bakg1j1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [80]:
test = pd.read_csv("test.csv", index_col='ID')
train = pd.read_csv("train.csv", index_col='ID')
train.head(2)

,url,title,label
ID,,,
0,m.kp.md,"Экс-министр экономики Молдовы - главе МИДЭИ, ц...",0
1,www.kp.by,Эта песня стала известна многим телезрителям б...,0


In [81]:
print(test.isnull().sum())
print(train.isnull().sum())

url      0
title    0
dtype: int64
url      0
title    0
label    0
dtype: int64


In [82]:
y_train = train['label']
X_train = train.drop(['label'], axis=1)

In [83]:
def preprosessing(text, stop_words, morph):
    words = text.translate(str.maketrans(string.punctuation, ' ' * len(string.punctuation))).lower().split()
    lemmatized_words = []
    for word in words:
        if word not in stop_words:
            lemmatized_word = morph.parse(word)[0].normal_form
            lemmatized_words.append(lemmatized_word)
    return " ".join(lemmatized_words)
    

In [84]:
stop_words = set(stopwords.words("russian")) | set(stopwords.words("english"))
morph = pymorphy2.MorphAnalyzer()

X_train['title'] = X_train['title'].apply(lambda x: preprosessing(x, stop_words, morph))
#X_train['url'] = X_train['url'].apply(lambda x: preprosessing(x, stop_words, morph))
test['title'] = test['title'].apply(lambda x: preprosessing(x, stop_words, morph))
#test['url'] = test['url'].apply(lambda x: preprosessing(x, stop_words, morph))

In [85]:
preprocessor = ColumnTransformer(
    transformers=[
        ('url', TfidfVectorizer(analyzer="char_wb", ngram_range=(2, 4), min_df=3, max_df=0.75), 'url'),
        ('title', CountVectorizer(), 'title')
    ])

In [86]:
X_train_preprocessed = preprocessor.fit_transform(X_train)
test_preprocessed = preprocessor.transform(test)

In [87]:
model = LogisticRegression(C=0.25, solver='lbfgs', class_weight='balanced', max_iter=2000)
model.fit(X_train_preprocessed, y_train)
y_pred = model.predict(test_preprocessed)

In [88]:
submission = pd.DataFrame({'ID':test.reset_index()['ID'], 'label':y_pred})
submission.head()

,ID,label
0,135309,0
1,135310,0
2,135311,0
3,135312,1
4,135313,0


In [89]:
submission.to_csv('submission.csv', index=False)